Runtime -> Change runtime type -> Hardware accelerator -> choose GPU

In [0]:
#install the adequate version of TensorFlow
!pip3 install tensorflow-gpu==1.13.1

In [0]:
#install library for yolov3 utilisation
!pip3 install imageai --upgrade

In [0]:
#Download a pretrained version of a yolov3 model
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

In [0]:
#unzip the datset of plates
!unzip plates.zip

In [0]:
#trains a YOLOv3 detection model
#data_ directory is the directory where our Pascal VOC dataset is saved ("plates" in this case)
#object_names_array is the list of objects we are detecting, i.e, the labels in the dataset
#num_experiments is the number of epochs we do
#batch_size is the batch size used
#train_from_pretrained_model is the path to model from which we start the training

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="plates")
trainer.setTrainConfig(object_names_array=["LP"], batch_size=4, num_experiments=2, train_from_pretrained_model="plates/models/detection_model-ex-002--loss-0007.222.h5")
trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.81
Anchor Boxes generated.
Detection configuration saved in  plates/json/detection_config.json
Training on: 	['LP']
Training with Batch Size:  4
Number of Experiments:  2
Training with transfer learning from pretrained Model


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/2
456/456 [==============================] - 503s 1s/step - loss: 5.2914 - yolo_layer_7_loss: 1.0471 - yolo_layer_8_loss: 1.8413 - yolo_layer_9_loss: 2.4029 - val_loss: 2.2997 - val_yolo_layer_7_loss: 0.6518 - val_yolo_layer_8_loss: 1.7096 - val_yolo_layer_9_loss: 2.0167
Epoch 2/2
456/456 [==============================] - 423s 928ms/step - loss: 4.7402 - yolo_layer_7_loss: 0.7489 - yolo_layer_8_loss: 1.7989 - yolo_layer_9_loss: 2.1923 - val_loss: 2.3365 - val_yolo_layer_7_loss: 0.5851 - val_yolo_layer_8_loss: 1.5482 - val_yolo_layer_9_loss: 2.1256


In [0]:
#Evaluates the models we have
#model_path is the path to where the models were saved
#json_path is the path to the detection_config.json file generated in the previous cell. It contains the anchors calculated
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="plates")
trainer.evaluateModel(model_path="plates/models", json_path="plates/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)


In [169]:
#Usage of the model for prediction
#input_image is the path to the image where the detection is done
#output_image_path is the path to the saved image with the detections done 
from imageai.Detection.Custom import CustomObjectDetection

input_image="voiture2.jpg"
output_image_path="voiture2-detected.jpg"

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("drive/My Drive/Colab Notebooks/plates/models/detection_model-ex-001--loss-0005.291.h5") 
detector.setJsonPath("drive/My Drive/Colab Notebooks/plates/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image, output_image_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


LP  :  53.26648950576782  :  [70, 113, 149, 124]


In [0]:
#Getting localisations for the plates in the image
import numpy as np
import cv2

def get_boxes(detections):
  boxes = []
  for detection in detections:
    boxes.append(detection["box_points"])
  boxes = np.array(boxes)
  return boxes

def cropped_plates(img_path, boxes, aug0=40,aug1=40,aug2=40,aug3=40):
  #increase boxes to deal with prediction's imprecision
  aug_boxes = np.array([-int(0.*aug0),int(aug1),-int(aug2),int(0.*aug3)])
  boxes += aug_boxes
  
  cropped_images = []
  img = cv2.imread(img_path) 
  cv2.resize(img, (416,416))
  for box in boxes:
    #trying to fix bug of ImageAI
    # height, width = img.shape[:2]
    # for i in range(4):
    #   if i<2:
    #     box[i]*=width/416.
    #   else:
    #     box[i]*=height/416.
    cropped = img[int(box[2]):int(box[3]),int(box[0]):int(box[1])]
    print(box)
    cropped_images.append(cropped)
  
  return cropped_images

#prints for debugging reasons
# boxes = get_boxes(detections)
# crp = cropped_plates(input_image,boxes)
# boxes
# cv2.imwrite('socorr.jpg', crp[0])



In [0]:
#API for reading text from image
!apt-get install tesseract-ocr

In [0]:
#Install necessary libraries
!pip install pillow
!pip install pytesseract
!pip install opencv-python

In [203]:
import cv2
import os
import pytesseract
import numpy as np


#INTEGRATION FROM DETECTION TO READING
#get the boxes from the detection
boxes = get_boxes(detections)
crpd_plates = cropped_plates(img_path=input_image, boxes=boxes)
if not os.path.exists('cropped_images'):
        os.makedirs('cropped_images')

# ipath=os.getcwd+"\\0.jpg"
def get_string(img_path, output_dir,img):
    # Extract the file name without the file extension
    file_name = os.path.basename(img_path).split('.')[0]
    file_name = file_name.split()[0]

    # Create a directory for outputs
    output_path = os.path.join(output_dir, file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return file_name, output_path, img

# img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

def noise_removal(img):
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    # Apply blur to smooth out the edges
    img = cv2.GaussianBlur(img, (5, 5), 0)
    
    return img

def binarization(img):
    # Apply threshold to get image with only b&w (binarization)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    return img
def binarization_gaussian(img):
    img = cv2.adaptiveThreshold(img,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11,2)
    return img

def save_result(img, output_path, file_name):
    # Save the filtered image in the output directory
    save_path = os.path.join(output_path, file_name + "_filter.jpg")
    cv2.imwrite(save_path, img)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(img, lang="eng")
    return result

#loop over the cropped images containing the plates
for i in range(len(crpd_plates)):
  img = crpd_plates[i]
  img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
  _, output_path, img = get_string('shot2.jpg', os.getcwd(),img) 
  
  img1 = noise_removal(img)
  # cv2.imwrite('socorro1.jpg', img1)
  img2 = binarization_gaussian(img1)
  # cv2.imwrite('socorro2.jpg', img2)
  
  result = save_result(img2,output_path, "plate_numbers")
  if result=='':
    img1 = 255-img1
    img2 = binarization_gaussian(img1)
    # cv2.imwrite('socorr.jpg', img2)
  
    result = save_result(img1,output_path, "plate_numbers")
# print(result)



[ 70 153 109 124]


In [204]:
result

'mA 737 AT i'